Import necessary packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from statsmodels.formula.api import ols

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
conn = sqlite3.connect('../../data/processed/main.db')
query = '''SELECT * FROM df_main'''
df_main = pd.read_sql(query, conn, index_col='index').reset_index(drop=True)

In [3]:
df_main.head(3)

,SalePrice,Parcel_ID,NbrLivingUnits,Stories,BldgGrade,SqFt1stFloor,SqFtHalfFloor,SqFt2ndFloor,SqFtUpperFloor,SqFtUnfinFull,SqFtUnfinHalf,SqFtTotLiving,SqFtTotBasement,SqFtFinBasement,FinBasementGrade,SqFtGarageBasement,SqFtGarageAttached,DaylightBasement,SqFtOpenPorch,SqFtEnclosedPorch,SqFtDeck,HeatSystem,HeatSource,BrickStone,ViewUtilization,Bedrooms,BathHalfCount,Bath3qtrCount,BathFullCount,FpSingleStory,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,Condition,SaleInstrument,SaleWarning,forced_air_heating
0,445000.0,010050-0380,1,1.0,7,1480,0,0,0,0,0,1480,0,0,0,0,440,,80,0,120,5,2,0.0,,3,0,0,2,1,0,0,0,1994,0,5,3,,forced
1,188500.0,017900-0315,1,1.0,5,550,0,0,0,0,0,550,0,0,0,0,0,,0,0,0,1,1,0.0,,2,0,1,0,0,0,0,0,1942,0,3,3,,non-forced
2,730000.0,018800-0095,1,2.0,9,660,0,595,0,0,0,1540,380,285,8,0,0,Y,0,0,155,1,3,0.0,Y,3,1,1,1,0,1,0,0,1984,0,3,3,,non-forced


In [4]:
df_main.columns

Index(['SalePrice', 'Parcel_ID', 'NbrLivingUnits', 'Stories', 'BldgGrade',
       'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor', 'SqFtUpperFloor',
       'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving', 'SqFtTotBasement',
       'SqFtFinBasement', 'FinBasementGrade', 'SqFtGarageBasement',
       'SqFtGarageAttached', 'DaylightBasement', 'SqFtOpenPorch',
       'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem', 'HeatSource',
       'BrickStone', 'ViewUtilization', 'Bedrooms', 'BathHalfCount',
       'Bath3qtrCount', 'BathFullCount', 'FpSingleStory', 'FpMultiStory',
       'FpFreestanding', 'FpAdditional', 'YrBuilt', 'YrRenovated', 'Condition',
       'SaleInstrument', 'SaleWarning', 'forced_air_heating'],
      dtype='object')

In [5]:
sqft_total = df_main[['SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor']].sum(axis=1)
sqft_total

0        1480
1         550
2        1255
3        1390
4        1350
         ... 
18138    1290
18139    3220
18140     960
18141    2010
18142     730
Length: 18143, dtype: int64

In [6]:
# df_resbldg['porch_both'] = (df_resbldg.SqFtOpenPorch > 0) & (df_resbldg.SqFtEnclosedPorch > 0)
# df_resbldg['porch_none'] = (df_resbldg.SqFtOpenPorch == 0) & (df_resbldg.SqFtEnclosedPorch == 0)
# df_resbldg['porch_closed'] = df_resbldg.SqFtEnclosedPorch > 0
# df_resbldg['porch_open'] = df_resbldg.SqFtOpenPorch > 0

In [7]:
# from custom_functions import one_hot
# from custom_functions import get_lookups

# heating = one_hot(df_resbldg['HeatSystem'], name_lookup = get_lookups(108, df_lookup), prefix='heat')
# df_resbldg = pd.concat([df_resbldg, heating], axis=1)

In [8]:
df_main['HeatSystem'].value_counts()

aggregate_heating_systems = lambda x: 'forced' if x==5 else 'non-forced'
binarize_heating = lambda x: 1 if x == 'forced' else 0

df_main['forced_air_heating'] = df_main.HeatSystem.apply(aggregate_heating_systems)
df_main['forced_air_heating_bin'] = df_main.forced_air_heating.apply(binarize_heating)


df_main.to_sql('df_main', conn, if_exists='replace')
#conn.close()